# Gold Standards registry
https://registry.goldstandard.org/projects?q=&page=3&project_types=22

In [57]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

### Load data

In [58]:
df = pd.read_csv("../input/Gold_Standards/GSF Registry Projects Export 2025-02-13.csv")
df.head(1)

,GSID,Project Name,Project Developer Name,Status,Sustainable Development Goals,Project Type,Country,Description,Estimated Annual Credits,Methodology,Size,Programme of Activities,POA GSID
0,12587,Parry Agro Industries C Cubed Tea and Rubber A...,Murali Balan Padikkal,Listed,"11,3,15,12,13,8,7,5,4",A/R,India,The C Cubed project combines tea and rubber ag...,21000,Afforestation/Reforestation GHG Emissions Redu...,Large Scale,Standalone,NaN


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   GSID                           71 non-null     int64  
 1   Project Name                   71 non-null     object 
 2   Project Developer Name         71 non-null     object 
 3   Status                         71 non-null     object 
 4   Sustainable Development Goals  71 non-null     object 
 5   Project Type                   71 non-null     object 
 6   Country                        71 non-null     object 
 7   Description                    71 non-null     object 
 8   Estimated Annual Credits       71 non-null     int64  
 9   Methodology                    71 non-null     object 
 10  Size                           71 non-null     object 
 11  Programme of Activities        71 non-null     object 
 12  POA GSID                       8 non-null      float

### Harmonize nomenclature

In [60]:
filtered_columns = ["GSID", "Country", "Description"]
gdf = df[filtered_columns].copy()

In [61]:
gdf['url'] = 'https://registry.goldstandard.org/projects?q=&page=3&project_types=22'
gdf['host_name'] = 'Gold Standards registry'

In [62]:
columns_rename_mapping = {

    'GSID': 'project_id_reported',
    'Description':'project_description_reported',
    'Country':'country',
}
gdf.rename(columns=columns_rename_mapping, inplace=True)

In [63]:
gdf = gdf.assign(project_description_reported=None,site_sqkm=None,species_count_reported=None, species_planted_reported=None, survival_rate_reported=None,trees_planted_reported=None,planting_date_reported=None,geometry=None,site_id_reported=None,)
gdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   project_id_reported           71 non-null     int64 
 1   country                       71 non-null     object
 2   project_description_reported  0 non-null      object
 3   url                           71 non-null     object
 4   host_name                     71 non-null     object
 5   site_sqkm                     0 non-null      object
 6   species_count_reported        0 non-null      object
 7   species_planted_reported      0 non-null      object
 8   survival_rate_reported        0 non-null      object
 9   trees_planted_reported        0 non-null      object
 10  planting_date_reported        0 non-null      object
 11  geometry                      0 non-null      object
 12  site_id_reported              0 non-null      object
dtypes: int64(1), object(12

### Save it

In [64]:
gdf = gpd.GeoDataFrame(gdf)


gdf.to_file('../midsave/gold_standards.gpkg', driver='GPKG')

/Users/angela/Library/Python/3.9/lib/python/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
